In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt # showing and rendering figures
# io related
from skimage.io import imread
import os
from glob import glob
# not needed in Kaggle, but required in Jupyter
%matplotlib inline 

import cv2
from tensorflow.keras.callbacks import ModelCheckpoint,CSVLogger
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Activation
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.utils import plot_model
from tensorflow.keras.datasets import cifar10
from tensorflow.keras import optimizers
from tensorflow.keras import regularizers

In [2]:
from keras.models import Sequential
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import add
from tensorflow.keras.layers import AveragePooling2D
import sklearn.metrics as metrics
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau



Using TensorFlow backend.


In [15]:
import os
os.chdir(r'D:\Intelligent Systems\Pattern Recognition Systems\PR-project\images\images\fashion')
df = pd.read_csv('data_dresses.csv',sep=';')

In [16]:
df.head()


,id,img_path,attributes,brand,category,color,img_url,model_img_urls,name,product_url,sleeve_length,length,neckline,pattern,fit,occasion
0,05.803.82.8357_595932,dresses/05.803.82.8357_595932.jpg,"Normaler Träger, Rundhals-Ausschnitt, Taillier...",s.Oliver RED LABEL,dresses,black,https://cdn.aboutstatic.com/file/d34a04c09625b...,https://cdn.aboutstatic.com/file/be94104d68a1b...,Kleid,https://www.aboutyou.de/p/s-oliver-red-label/k...,short,knee,round,unicolors,tight,NaN
1,05.807.82.6853_46B132,dresses/05.807.82.6853_46B132.jpg,"Abgesteppter Saum/Kante, Ausgestellter Schnitt...",s.Oliver RED LABEL,dresses,pink,https://cdn.aboutstatic.com/file/8e2503d650365...,https://cdn.aboutstatic.com/file/89d75c9971649...,Kleid,https://www.aboutyou.de/p/s-oliver-red-label/k...,sleeveless,short,back,floral,tight,NaN
2,05.807.82.7936_59G332,dresses/05.807.82.7936_59G332.jpg,"Abgesteppter Saum/Kante, Ausgestellter Schnitt...",s.Oliver RED LABEL,dresses,blue,https://cdn.aboutstatic.com/file/cc792eac6cfd9...,https://cdn.aboutstatic.com/file/42b431d51042c...,Kleid,https://www.aboutyou.de/p/s-oliver-red-label/k...,short,short,deep,stripes,tight,NaN
3,100174,dresses/100174.jpg,"Damenkleid von Daniel Hechter, Leichtes Materi...",Daniel Hechter,dresses,black,https://img.fidcdn.net/r17/product/daniel-hech...,https://img.fidcdn.net/r17/product/daniel-hech...,Kleid mit Rundhalsausschnitt und kurzem Schlitz,https://www.fashionid.de/daniel-hechter/damen-...,NaN,NaN,NaN,NaN,loose,NaN
4,1001751002,dresses/1001751002.jpg,"Rundhals-Ausschnitt, Abgesteppter Saum/Kante, ...",TOM TAILOR DENIM,dresses,blue,https://cdn.aboutstatic.com/file/486c43af4548d...,https://cdn.aboutstatic.com/file/d22b4007739a0...,Kleid,https://www.aboutyou.de/p/tom-tailor-denim/kle...,long,3-4,round,NaN,normal,NaN


In [17]:
df.sample(frac=1)

,id,img_path,attributes,brand,category,color,img_url,model_img_urls,name,product_url,sleeve_length,length,neckline,pattern,fit,occasion
3331,9729130,dresses/9729130.jpg,"Damen Kleid von REVIEW, Krepp, Taillierter Sch...",REVIEW,dresses,black,https://img.fidcdn.net/r17/product/review-klei...,https://img.fidcdn.net/r17/product/review-klei...,Kleid aus Krepp mit Sternenmuster,https://www.fashionid.de/review/damen-kleid-au...,NaN,NaN,round,NaN,tight,NaN
15196,Y0321C017-Q11,dresses/Y0321C017-Q11.jpg,"Material & Pflegehinweise, Material Oberstoff:...",someday.,dresses,black,https://mosaic04.ztat.net/vgs/media/packshot/p...,https://mosaic03.ztat.net/vgs/media/pdp-zoom/Y...,QUASTI - Freizeitkleid,NaN,long,knee,round,unicolors,loose,casual
9791,KL221C087-Q11,dresses/KL221C087-Q11.jpg,"Material & Pflegehinweise, Material Oberstoff:...",King Louie,dresses,black,https://mosaic01.ztat.net/vgs/media/packshot/p...,https://mosaic01.ztat.net/vgs/media/pdp-zoom/K...,GINA DRESS MADELIN - Freizeitkleid,NaN,short,knee,NaN,floral,tight,casual
11518,NAK1459002000001,dresses/NAK1459002000001.jpg,"Breiter Bund, Taillierter Schnitt, Teilungsnäh...",naketano,dresses,black,https://cdn.aboutstatic.com/file/00616193ca9ca...,https://cdn.aboutstatic.com/file/e41ec948c5b45...,Sweatshirt-Kleid,https://www.aboutyou.de/p/naketano/sweatshirt-...,long,short,NaN,unicolors,tight,NaN
5998,A0Q21C046-J11,dresses/A0Q21C046-J11.jpg,"Material & Pflegehinweise, Material Oberstoff:...",AllSaints,dresses,pink,https://mosaic03.ztat.net/vgs/media/packshot/p...,https://mosaic03.ztat.net/vgs/media/pdp-zoom/A...,ANZA DRESS - Strickkleid,NaN,sleeveless,knee,v,print,tight,NaN
12382,PC721C040-Q11,dresses/PC721C040-Q11.jpg,"Material & Pflegehinweise, Material Oberstoff:...",Opus,dresses,black,https://mosaic04.ztat.net/vgs/media/packshot/p...,https://mosaic03.ztat.net/vgs/media/pdp-zoom/P...,WERIA GRAPHIC - Freizeitkleid,NaN,sleeveless,short,round,print,normal,casual
8343,EV421C0K3-Q11,dresses/EV421C0K3-Q11.jpg,"Material & Pflegehinweise, Material Oberstoff:...",Even&Odd,dresses,black,https://mosaic03.ztat.net/vgs/media/packshot/p...,https://mosaic03.ztat.net/vgs/media/pdp-zoom/E...,Jerseykleid,NaN,long,short,NaN,floral,tight,NaN
12582,PU121C00S-Q11,dresses/PU121C00S-Q11.jpg,"Material & Pflegehinweise, Material Oberstoff:...",Puma,dresses,black,https://mosaic03.ztat.net/vgs/media/packshot/p...,https://mosaic03.ztat.net/vgs/media/pdp-zoom/P...,DRESS - Etuikleid,NaN,long,short,back,print,tight,NaN
4217,9799525,dresses/9799525.jpg,Damen Etuikleid von Christian Berg Woman Selec...,Christian Berg Woman Selection,dresses,blue,https://img.fidcdn.net/r17/product/christian-b...,https://img.fidcdn.net/r17/product/christian-b...,Etuikleid mit floralem Muster,https://www.fashionid.de/christian-berg-woman-...,sleeveless,NaN,round,floral,tight,NaN
13027,SAM0360002000001,dresses/SAM0360002000001.jpg,"Breiter Träger, Rundhals-Ausschnitt, Abgestepp...",Samsoe & Samsoe,dresses,blue,https://cdn.aboutstatic.com/file/fabc1f928966e...,https://cdn.aboutstatic.com/file/27a47990b3ee0...,Midikleid 'Mentha',https://www.aboutyou.de/p/samsoe-und-samsoe/mi...,sleeveless,knee,round,print,normal,NaN


In [18]:
df.groupby(df['sleeve_length']).count()

,id,img_path,attributes,brand,category,color,img_url,model_img_urls,name,product_url,length,neckline,pattern,fit,occasion
sleeve_length,,,,,,,,,,,,,,,
half,2796,2796,2796,2796,2796,2796,2796,2792,2796,1469,2466,2268,2389,2762,798
long,2548,2548,2548,2548,2548,2548,2548,2541,2548,1456,2360,1672,2214,2523,609
short,3026,3026,3026,3026,3026,3026,3026,3018,3026,1557,2740,2461,2602,2990,918
sleeveless,4959,4959,4959,4959,4959,4959,4959,4946,4959,3210,3967,3893,4024,4565,1477


In [19]:
df[df['sleeve_length']== np.nan].shape

(0, 16)

In [20]:
df['sleeve_length'].unique()

array(['short', 'sleeveless', nan, 'long', 'half'], dtype=object)

In [21]:
df = df.groupby(df['sleeve_length']).head(2500)

In [22]:
df = df.dropna(axis=0, subset=['sleeve_length']).sample(frac=1)
df = df.reset_index(drop=True)

In [23]:
df.groupby(df['sleeve_length']).count()

,id,img_path,attributes,brand,category,color,img_url,model_img_urls,name,product_url,length,neckline,pattern,fit,occasion
sleeve_length,,,,,,,,,,,,,,,
half,2500,2500,2500,2500,2500,2500,2500,2496,2500,1365,2175,2024,2111,2467,688
long,2500,2500,2500,2500,2500,2500,2500,2494,2500,1436,2313,1644,2168,2475,595
short,2500,2500,2500,2500,2500,2500,2500,2498,2500,1364,2218,2014,2110,2466,712
sleeveless,2500,2500,2500,2500,2500,2500,2500,2494,2500,2155,1593,1950,1764,2127,558


In [24]:
trlen = int(len(df)*0.75)
tslen = len(df)-trlen
print(trlen,tslen)


7500 2500


In [25]:
#shape = (200,200)  
tsDat = [] 
trDat = []
trlabels = []
tslabels = []
for i in range(int(len(df))):
    if i <trlen:
        img = cv2.imread(df['img_path'][i])
        #img = cv2.resize(img,shape)
        trDat.append(img)
        trlabels.append(df['sleeve_length'][i])
    else:
        img = cv2.imread(df['img_path'][i])
        #img = cv2.resize(img,shape)
        tsDat.append(img)
        tslabels.append(df['sleeve_length'][i])
    
    
trLbl = pd.get_dummies(trlabels).values    
trlabelname = pd.get_dummies(trlabels).columns
tsLbl = pd.get_dummies(tslabels).values    
tslabelname = pd.get_dummies(tslabels).columns


In [26]:
trDat = np.array(trDat)
tsDat = np.array(tsDat)

In [27]:
dfc = pd.DataFrame({'key': np.array(tslabels),
                   'data': range(len(tslabels))}, columns=['key', 'data'])
dfc.groupby('key').count()


,data
key,
half,629
long,671
short,576
sleeveless,624


imagenumber =13
print(images[imagenumber].shape)
#
print(labels[imagenumber].shape)
print(labelname[int(np.where(labels[imagenumber]==1)[0])])
RGB_img = cv2.cvtColor(images[imagenumber], cv2.COLOR_BGR2RGB)
plt.imshow(RGB_img)


In [28]:
imgrows         = trDat[0].shape[0]
imgclms         = trDat[0].shape[1]
channel         = trDat[0].shape[2]
num_classes     = len(np.unique(tslabels))

In [29]:
print(imgrows,imgclms,channel,num_classes)

256 256 3 4


In [30]:
optmz       = optimizers.RMSprop(lr=0.0001)                                     # Step 1

                                                                                # Step 3
def createModel():
    inputs = Input(shape=(imgrows,imgclms,channel))
    x = Conv2D(32,(3,3),padding='same')(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(0.25)(x)
    x = Conv2D(32,(3,3),padding='same')(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(pool_size=(2,2))(x)
    x = Flatten()(x)
    x = Dense(64,activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(num_classes,activation='softmax')(x) 
    
    model = Model(inputs=inputs,outputs=x) 

    model.compile(loss='categorical_crossentropy', 
                  optimizer=optmz, 
                  metrics=['accuracy'])


    return model

                                                                                # Step 4
model       = createModel() # This is meant for training
modelGo     = createModel() # This is used for final testing

model.summary()                                                                 # Step 5

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 256, 256, 32)      896       
_________________________________________________________________
batch_normalization (BatchNo (None, 256, 256, 32)      128       
_________________________________________________________________
activation (Activation)      (None, 256, 256, 32)      0         
_________________________________________________________________
dropout (Dropout)            (None, 256, 256, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 256, 256, 32)      9248      
_________________________________________________________________
activation_1 (Activation)    (None, 256, 256, 32)      0     

In [31]:
modelname       = 'sleeve_length_pred_conv_51'                                                                                # Step 1
folderpath      = 'D:\\Intelligent Systems\\Pattern Recognition Systems\\PR-project\\jupytermodel\\'
filepath        = folderpath + modelname + ".hdf5"
checkpoint      = ModelCheckpoint(filepath, 
                                  monitor='val_accuracy', 
                                  verbose=0, 
                                  save_best_only=True, 
                                  mode='max')
early = EarlyStopping(monitor="val_loss", 
                      mode="min", 
                      patience=8, restore_best_weights=True)
csv_logger      = CSVLogger(folderpath+modelname +'.csv')                       # Step 2
callbacks_list  = [checkpoint,csv_logger,early]                                       # Step 3

print("Callbacks created:")
print(callbacks_list[0])
print(callbacks_list[1])
print('')
print("Path to model:", filepath)
print("Path to log:  ", folderpath+modelname+'.csv')

Callbacks created:

Path to model: D:\Intelligent Systems\Pattern Recognition Systems\PR-project\jupytermodel\sleeve_length_pred_conv_51.hdf5
Path to log:   D:\Intelligent Systems\Pattern Recognition Systems\PR-project\jupytermodel\sleeve_length_pred_conv_51.csv


In [32]:
history = model.fit(trDat,                    # Training data
          trLbl,                            # Training label
          validation_data=(tsDat, tsLbl),   # Validation data and label
          epochs=20,                       # The amount of epochs to be trained
          batch_size=20,                   
          shuffle=True                     # To shuffle the training data
          ,callbacks=callbacks_list         # Callbacks to execute the checkpoints
         )

Epoch 1/20
375/375 [==============================] - 285s 760ms/step - loss: 2.2968 - accuracy: 0.2520 - val_loss: 1.3862 - val_accuracy: 0.2292
Epoch 2/20
 25/375 [=>............................] - ETA: 4:05 - loss: 1.3862 - accuracy: 0.2740

KeyboardInterrupt: 

In [35]:
device = cuda.get_current_device()
device.reset()



NameError: name 'cuda' is not defined

In [ ]:
#VIsualizing the training performance
plt.figure(figsize=(12, 8))

plt.subplot(2, 2, 1)
plt.plot(history.history['loss'], label='Loss')
plt.plot(history.history['val_loss'], label='val_Loss')
plt.legend()
plt.title('Loss evolution')

plt.subplot(2, 2, 2)
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.legend()
plt.title('Accuracy evolution')

In [ ]:
from tensorflow.keras.utils import plot_model
plot_model(model, 
           to_file='CNN_model_sleevelength_5.png', 
           show_shapes=True, 
           show_layer_names=False,
           rankdir='TB')

In [ ]:
modelGo.load_weights(filepath)
modelGo.compile(loss='categorical_crossentropy', 
                optimizer=optmz, 
                metrics=['accuracy'])

predicts    = modelGo.predict(tsDat)                                            # Step 2
print("Prediction completes.")

In [ ]:
predout     = np.argmax(predicts,axis=1)
testout     = np.argmax(tsLbl,axis=1)

testScores  = metrics.accuracy_score(testout,predout)                           # Step 3

                                                                                # Step 4
print("Best accuracy (on testing dataset): %.2f%%" % (testScores*100))
print(metrics.classification_report(testout,
                                    predout,
                                    target_names=tslabelname,
                                    digits=4))

In [ ]:
confusion   = metrics.confusion_matrix(testout,predout)
print(confusion)

In [ ]:
model.save(filepath)